## 한글 NLP

### 어렵게 하는 요소
<img src=https://blog.kakaocdn.net/dn/b5wdPz/btrGlUj3U2X/InTjCFDmoi9kzmoTC5gIk1/img.png width=1000>

### 한글 형태소 분석
- 어절 분석과의 차이: 어절 분석은 공백 단위로 구분하는 거고, 형태소는 의미갖는 최소 단위로 구분.
<br>
<img src=https://blog.kakaocdn.net/dn/pUcG2/btrGgKcMKOg/3NPaCuQit6MS4eW91TSSx0/img.png width=1000>

### KoNLPy: 한글 형태소 분석 모듈
<img src=https://blog.kakaocdn.net/dn/H5yAE/btrGlUEmEfz/jA3Bmdpwz8keHklCxJzrkk/img.png width=1000>

### KoNLPy 설치
https://konlpy.org/ko/latest/install/#id1
<img src=https://blog.kakaocdn.net/dn/pB4G9/btrGlg8Fg0o/o9VMUAsE5WgMVClLy6ItTk/img.png width=700>

## 네이버 영화 평점 분석

### 데이터 로드
https://github.com/e9t/nsmc
<br><img src=https://blog.kakaocdn.net/dn/IfY2h/btrGqJR3M0R/078osPt0WJOaD0khNJrar1/img.png width=1000>

In [1]:
from konlpy.tag import Twitter
from konlpy.tag import Okt
from konlpy.tag import Kkma
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 컬럼 분리 문자 \t 및 한글 인코딩 cp949 적용
# label 1은 긍정적인 평가, 0은 부정적인 평가
train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [2]:
# 긍정적인 평가(1)와 부정적인 평가(0) 빈도
train_df['label'].value_counts( )

0    75173
1    74827
Name: label, dtype: int64

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [4]:
# document가 Null인 5개 데이터 확인
train_df[train_df.document.isna()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


### 데이터 전처리
1. document 컬럼 fillna(' ')
2. document 컬럼 숫자 1개 이상 -> 공백으로
3. id 컬럼 drop

In [5]:
import re

# document 컬럼에 Null 5개 존재 -> 공백으로 채워버리기
train_df = train_df.fillna(' ')

# 정규 표현식을 이용하여 숫자를 공백으로 변경 -> 숫자가 1개 이상인 경우 공백으로
train_df['document'] = train_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )
train_df.drop('id', axis=1, inplace=True) # id 컬럼 drop

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  150000 non-null  object
 1   label     150000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [7]:
# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('ratings_test.txt', sep='\t')
# 테스트 데이터도 마찬가지로 전처리
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )
test_df.drop('id', axis=1, inplace=True)

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  50000 non-null  object
 1   label     50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


### 형태소 토큰화

In [15]:
from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

tw_tokenizer('아버지가방에들어가신다.')

['아버지', '가방', '에', '들어가신다', '.']

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2)
# tokenizer 사용하게 되면, 라인 바이 라인으로 문서(or 문장)마다 토큰화 적용된다.
# max_df=0.9 - 단어가 전체 문서중 90퍼센트의 문서에서 나온 단어까지만 허용. 90퍼센트 이상의 문서에서 나온 단어라면 제외
# min_df=3 - 단어가 전체 문서중 3개 이상의 문서에서 나온 단어부터 허용
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [12]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   46.3s finished


{'C': 3.5} 0.8593


In [13]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.86172
